In [1]:
import pandas as pd
import matplotlib as mp
import numpy as np
from datetime import timedelta
import datetime as dt

%matplotlib inline

#Root de archivos de datos
root_dir = '../data/'

#Lee menos data si es testing
is_testing = False

# Cantidad minima de aparicion de equipos en un dataframe
min_devices = 5

# Identificador unico de equipos
device_uuid = ['ref_hash']

# Ventanas de tiempo
windows = pd.DataFrame({
    'begin_date': [dt.datetime(2019,4,18),dt.datetime(2019,4,21),dt.datetime(2019,4,24)],
    'window_nr':[1,2,3]
})

# Armado de Features

Se utilizan 3 ventanas de tiempo y se procesan los dataframes para obtener features de esas 3 ventanas por separado. 

Como resultado se obtiene un dataset de features mas grande cuyos datos no se solapan en tiempo. Por lo que son validos para la prediccion. 

Se realiza la lectura y limpiado de los dataframes principales, con los mismos se realiza el filtrado y armado de los features. 

---
## 1. Lectura de DataFrames

In [2]:
#Optimizado para menos memoria
auction_dtypes = {
    'device_id': np.int64,
    'source_id': np.int8
}

auctions = pd.read_csv(root_dir + 'auctions.csv.gzip',
                       compression = 'gzip',
                       dtype = auction_dtypes,
                       usecols=list(auction_dtypes.keys()) + ['date'],
                       parse_dates = ['date'])
auctions.rename({'device_id':'ref_hash',
                 'date':'created'}, inplace=True, axis='columns')
auctions['n'] = 1

# Para hacer pruebas ocupando menos memoria se hace un sampleo aleatorio de la mitad del dataframe y se elimina el resto
if is_testing:
    auctions = auctions.sample(frac=.30)
    print("Is Testing, #records:", auctions['n'].sum())
else:
    print("Not Testing, #records:", auctions['n'].sum())

Not Testing, #records: 47409528


In [3]:
# Labels a submitir con las predicciones
to_predict = pd.read_csv(root_dir + 'target_competencia_ids.csv')
to_predict = to_predict.apply(lambda x: np.int64(x['ref_hash'][0:x['ref_hash'].find('_')]), axis='columns').drop_duplicates().to_frame()
to_predict.columns = ['ref_hash']

In [4]:
unique_hashes = pd.read_csv(root_dir+'unique_hashes.csv')

---
## 2. Arreglo de los datos

### Separo las semanas de entrenamiento

Se utiliza el siguiente metodo: 

1. Ventana del 18 al 20 inclusive (1) -> Predice valores entre el 21 y 24 (2)
2. Ventana del 21 al 23 inclusive (2) -> Predice valores entre el 24 y 26 (3)

In [5]:
#Agrego la ventana de tiempo
auctions = auctions.loc[auctions['ref_hash'].isin(unique_hashes['ref_hash'])]\
                   .sort_values(by='created')
auctions = pd.merge_asof(auctions,windows,left_on='created',right_on='begin_date').drop('begin_date', axis='columns')

### 2.1 Arreglo de Auctions

In [6]:
%%time
#Elimino los registros con menos de un minimo de entradas, ya que no hay mucho que predecir en estos casos
group = ['ref_hash','window_nr']
orig_count = auctions['n'].sum()
auctions = auctions.groupby(group, sort = False)\
                   .filter(lambda x: x['n'].sum() >= min_devices)\
                   .sort_values(by=group+['created'])
last_count = auctions['n'].sum()
print('Eliminados:', orig_count-last_count)

Eliminados: 736875
CPU times: user 10min 8s, sys: 17.9 s, total: 10min 26s
Wall time: 10min 22s


In [7]:
%%time
auctions['next_date'] = auctions.groupby(group, as_index = False, sort=False)['created'].transform(lambda x: x.shift(-1))
auctions = auctions.loc[(~auctions['next_date'].isnull())]
auctions['secs_to_next'] = (auctions['next_date'] - auctions['created']).transform(lambda x: round(x.total_seconds()))

CPU times: user 17min 15s, sys: 17.8 s, total: 17min 33s
Wall time: 19min 19s


In [8]:
auctions.head()

,created,ref_hash,source_id,n,window_nr,next_date,secs_to_next
9146132,2019-04-19 19:40:28.465866,41863526108385,8,1,1,2019-04-20 02:52:26.892880,25918
11162626,2019-04-20 02:52:26.892880,41863526108385,3,1,1,2019-04-20 02:59:02.509230,396
11194917,2019-04-20 02:59:02.509230,41863526108385,3,1,1,2019-04-20 03:06:01.675788,419
11229465,2019-04-20 03:06:01.675788,41863526108385,3,1,1,2019-04-20 03:08:57.388160,176
11244486,2019-04-20 03:08:57.388160,41863526108385,3,1,1,2019-04-20 03:11:26.463903,149


## Lectura y limpieza de otros dataframes

Esto se hace despues pues preparar Auctions requiere de toda la memoria posible libre

In [226]:
events_dtypes = {
    'event_id': np.int64,
    'ref_hash': np.int64,
    'application_id': np.int64,
    'attributed': np.bool,
    'device_os_version': 'object',
    'device_brand': 'object',
    'device_model': 'object',
    'device_city': 'object',
    'session_user_agent': 'object',
    'trans_id': 'category',
    'user_agent': 'object',
    'event_uuid': 'object',
    'carrier': 'object',
    'kind': 'object',
    'device_os': 'object',
    'wifi': np.bool,
    'connection_type': 'object',
    #'ip_address': np.int64,
    #'device_language': 'category'
}
events_cols = list(events_dtypes.keys()) + ['date']
events = pd.read_csv(root_dir + 'events.csv.gzip', 
                     compression='gzip',
                     dtype=events_dtypes,
                     usecols=events_cols,
                     parse_dates=['date'])

events.rename({'date':'created'}, inplace=True, axis='columns')
events['n'] = 1
# Para hacer pruebas ocupando menos memoria se hace un sampleo aleatorio de la mitad del dataframe y se elimina el resto
if is_testing:
    events = events.sample(frac=.30)
    print("Is Testing, #records:", events['n'].sum())
else:
    print("Not Testing, #records:", events['n'].sum())

Not Testing, #records: 7744581


In [227]:
installs_dtypes = {
    'application_id': np.int32,
    'ref_hash': np.int64, 
    #'click_hash':'category',
    'attributed': 'category',
    'implicit': 'category',
    #'device_countrycode': 'object', 
    'device_brand': 'object',
    'device_model': 'object', 
    'session_user_agent': 'object', 
    'user_agent': 'object', 
    'event_uuid':'object',
    'kind': 'object',
    'wifi': 'object', 
    'trans_id': 'object', 
    #'ip_address':'object', 
    'device_language': 'object'
}
install_cols = list(installs_dtypes.keys()) + ['created']
installs = pd.read_csv(root_dir + 'installs.csv.gzip', 
                       compression='gzip', 
                       usecols=install_cols,
                       dtype= installs_dtypes,
                      parse_dates=['created'])

installs['n'] = 1
print('#records:', installs['n'].sum())

#records: 481511


In [228]:
clicks_dtypes = {
    'advertiser_id': np.int64, 
    'action_id': np.float, 
    'source_id': np.int8, 
    'latitude' : np.float64, 
    'longitude': np.float64, 
    'wifi_connection': np.bool, 
    'carrier_id': 'object', 
    'trans_id': 'object',
    'os_minor':'object', 
    'agent_device' : 'object', 
    'os_major': 'object', 
    'specs_brand': 'category', 
    'brand': 'object',
    'timeToClick': np.float64, 
    'touchX': 'object', 
    'touchY': 'object', 
    'ref_hash':np.int64,
    'created' : 'object'
}
clicks = pd.read_csv(root_dir + 'clicks.csv.gzip', 
                     compression='gzip',
                     usecols=list(clicks_dtypes.keys()) + ['created'],
                     dtype=clicks_dtypes,
                     parse_dates=['created'])
clicks['n'] = 1
print('#records:', clicks['n'].sum())

#records: 64296


In [229]:
installs = installs.loc[installs['ref_hash'].isin(unique_hashes['ref_hash'])]
installs.sort_values(by='created',inplace=True)
installs = pd.merge_asof(installs,windows,left_on='created',right_on='begin_date').drop('begin_date', axis='columns')

In [230]:
clicks = clicks.loc[clicks['ref_hash'].isin(unique_hashes['ref_hash'])]
clicks['created'] = clicks['created'].dt.tz_localize(None)
clicks.sort_values(by='created',inplace=True)
clicks = pd.merge_asof(clicks,windows,left_on='created',right_on='begin_date').drop('begin_date', axis='columns')

In [231]:
events = events.loc[events['ref_hash'].isin(unique_hashes['ref_hash'])]
events.sort_values(by='created',inplace=True)
events = pd.merge_asof(events,windows,left_on='created',right_on='begin_date').drop('begin_date', axis='columns')

### Arreglo de installs

In [232]:
installs['kind'] = installs['kind'].str.replace(' ','_')
installs['kind'] = installs['kind'].str.replace('af_app_open ','af_app_opened')
installs['kind'] = installs['kind'].str.replace('af_app_opend','af_app_opened')
installs['kind'] = installs['kind'].str.lower()

In [233]:
installs.isnull().sum()


created                    0
application_id             0
ref_hash                   0
attributed                 0
implicit                   0
device_brand          204813
device_model           26871
session_user_agent     14828
user_agent            150111
event_uuid            377704
kind                  377704
wifi                  186016
trans_id              472140
device_language        27552
n                          0
window_nr                  0
dtype: int64

In [234]:
for i in ['device_brand','device_model','session_user_agent','user_agent','kind','wifi','device_language']:
    installs[i] = installs[i].fillna('unknown')
installs['device_brand'] = installs['device_brand'].astype('category')
installs['device_model'] = installs['device_model'].astype('category')
installs['session_user_agent'] = installs['session_user_agent'].astype('category')
installs['user_agent'] = installs['user_agent'].astype('category')
installs['kind'] = installs['kind'].astype('category')
installs['wifi'] = installs['wifi'].astype('category')
installs['device_language'] = installs['device_language'].astype('category')

In [235]:
installs.head()


,created,application_id,ref_hash,attributed,implicit,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,device_language,n,window_nr
0,2019-04-18 00:00:01.560,70,4432995619177048534,False,False,unknown,unknown,Apsalar-Postback,unknown,NaN,unknown,unknown,NaN,unknown,1,1
1,2019-04-18 00:00:01.851,70,5904733559638204455,False,False,unknown,unknown,Apsalar-Postback,unknown,NaN,unknown,unknown,NaN,unknown,1,1
2,2019-04-18 00:00:05.152,65,896373747754111825,False,True,3.083058605577787e+17,5.274185305862703e+18,http-kit/2.0,Dalvik/2.1.0 (Linux; U; Android 9; SM-G9650 Bu...,8c8af5e3-96e7-4a49-9f17-cafa7f300f2c,af_app_opened,false,NaN,6.977049253562486e+18,1,1
3,2019-04-18 00:00:05.589,27,3399210824535017892,False,False,unknown,6.794880020077885e+18,http-kit/2.0,trivago/216 CFNetwork/978.0.7 Darwin/18.5.0,NaN,unknown,true,NaN,5.221862722669226e+18,1,1
4,2019-04-18 00:00:06.795,339,1541425881979513687,False,False,unknown,6.794880020077885e+18,http-kit/2.0,TikTok/109005 CFNetwork/758.5.3 Darwin/15.6.0,NaN,unknown,true,NaN,7.528973756559112e+18,1,1


### Arreglo de Clicks

In [236]:
clicks.touchX = clicks.touchX.apply(lambda x: np.float64(x))
clicks.touchY = clicks.touchY.apply(lambda x: np.float64(x))
clicks['created'] = clicks['created'].dt.tz_localize(None)

In [237]:
for i in ['carrier_id','os_minor', 'agent_device', 'brand', 'os_major']:
    clicks[i] = clicks[i].fillna('unknown').astype('category')

In [238]:
clicks.isnull().sum()


advertiser_id          0
action_id          63835
source_id              0
created                0
latitude               0
longitude              0
wifi_connection        0
carrier_id             0
trans_id               0
os_minor               0
agent_device           0
os_major               0
specs_brand            0
brand                  0
timeToClick        25885
touchX             20428
touchY             20428
ref_hash               0
n                      0
window_nr          19730
dtype: int64

In [239]:
clicks.head()

,advertiser_id,action_id,source_id,created,latitude,longitude,wifi_connection,carrier_id,trans_id,os_minor,agent_device,os_major,specs_brand,brand,timeToClick,touchX,touchY,ref_hash,n,window_nr
0,1,NaN,1,2019-04-12 00:00:01.981,1.714386,0.871408,False,1.0,OHtntFaZBW28OcZvxtGT0rLBLqmccrM,1.5176438893491397e+18,unknown,5.131615556736863e+18,3576558787748411622,0.0,19.296,0.952,0.070,3455656761384645032,1,NaN
1,2,NaN,1,2019-04-12 00:00:29.662,1.712736,0.869157,True,24.0,XZZOyg3ZOEFw9MFeBJWF7dNUl7Dl8AQ,3.575963029724781e+18,unknown,5.754947116114108e+18,3576558787748411622,0.0,625.859,0.474,11.684,2751776655120553475,1,NaN
2,1,NaN,0,2019-04-12 00:00:53.624,1.714547,0.871535,False,13.0,ptDi6mZepLORp2kxILbv_g5WxOVgqHI,1.2885781261232225e+18,unknown,3.908390200756879e+18,71913840936116953,unknown,29.908,0.130,0.179,6931142988092289392,1,NaN
3,1,NaN,1,2019-04-12 00:01:07.438,1.837799,0.829634,True,23.0,MhX94IlfeUwwWATXfHlLHKGPwSepPv4,2.2387361390161664e+18,unknown,3.581232574980917e+18,3576558787748411622,unknown,377.566,0.446,0.510,2114792698570967449,1,NaN
4,1,NaN,1,2019-04-12 00:01:08.535,1.733375,0.908130,True,1.0,BubcDEi6KKiYNUoN3XZmHioWNYna5N0,5.310344816890522e+18,unknown,3.581232574980917e+18,3576558787748411622,unknown,141.725,0.747,0.022,3949216411894914793,1,NaN


### Arreglo de Events

In [240]:
for i in ['device_os_version','device_brand','device_model','device_city','session_user_agent','user_agent','carrier','kind','device_os','connection_type']:
    events[i] = events[i].fillna('unknown').astype('category')

In [241]:
events.isnull().sum()


created                     0
event_id                    0
ref_hash                    0
application_id              0
attributed                  0
device_os_version           0
device_brand                0
device_model                0
device_city                 0
session_user_agent          0
trans_id              7700153
user_agent                  0
event_uuid              29631
carrier                     0
kind                        0
device_os                   0
wifi                        0
connection_type             0
n                           0
window_nr                   0
dtype: int64

---
## 3. Armado de Features

A continuacion se comienzan a extraer los distintos features que formaran el set de entrenamiento. 
Se cruza cada ventana con los labels que se desean predecir. 

Para entrenar el set debe decidirse como utilizar los datos de las distintas ventanas. 

Se tomaran los datos en 2 ventanas diferentes y se validaran con dos ventanas diferentes, de esta forma no deberia haber problemas de solapamiento y se maximiza la cantidad de datos.

Para el set final puede armarse un set de labels con los valores de los ultimos 3 dias.

In [275]:
max_dates = pd.DataFrame({'w_max_date': [dt.datetime(2019,4,21),dt.datetime(2019,4,24),dt.datetime(2019,4,27)], 'window_nr' : [1,2,3]})

In [276]:
temp = unique_hashes
temp['window_nr'] = 1
devices = temp.copy()
temp['window_nr'] = 2
devices = devices.append(temp)
temp['window_nr'] = 3
devices = devices.append(temp)

In [277]:
devices['window_nr'].value_counts()

3    662110
2    662110
1    662110
Name: window_nr, dtype: int64

In [278]:
devices = devices.set_index(group)

In [279]:
sample = auctions.head(100)

In [280]:
sample.head()

,created,ref_hash,source_id,n,window_nr,next_date,secs_to_next
9146132,2019-04-19 19:40:28.465866,41863526108385,8,1,1,2019-04-20 02:52:26.892880,25918
11162626,2019-04-20 02:52:26.892880,41863526108385,3,1,1,2019-04-20 02:59:02.509230,396
11194917,2019-04-20 02:59:02.509230,41863526108385,3,1,1,2019-04-20 03:06:01.675788,419
11229465,2019-04-20 03:06:01.675788,41863526108385,3,1,1,2019-04-20 03:08:57.388160,176
11244486,2019-04-20 03:08:57.388160,41863526108385,3,1,1,2019-04-20 03:11:26.463903,149


### Tiempo promedio de arribos

Se desea saber el tiempo promedio entre arribos de los dispositivos a las encuestas.

In [281]:
%%time
time = auctions.groupby(group,sort=False)['secs_to_next'].mean()\
               .rename('secs_to_next_mean')\
               .transform(lambda x: round(x))

CPU times: user 2.77 s, sys: 1.08 s, total: 3.85 s
Wall time: 3.85 s


In [282]:
devices = devices.merge(time, how='left', left_index=True, right_index=True)

In [283]:
devices.head()

,,secs_to_next_mean
ref_hash,window_nr,
2564673204772915246,1,765.0
4441121667607578179,1,111.0
7721769811471055264,1,281.0
6416039086842158968,1,NaN
1258642015983312729,1,8365.0


### Tiempo desde ultima aparicion

In [284]:
%%time
time = auctions.groupby(group,sort=False)\
               .tail(1)\
               .merge(max_dates, on='window_nr', how='left')\
               .set_index(group)

time = (time['w_max_date'] - time['created']).transform(lambda x: round(x.total_seconds())).rename('secs_since_last_arrival')

CPU times: user 13.7 s, sys: 2.04 s, total: 15.7 s
Wall time: 15.7 s


In [285]:
time.head()

ref_hash         window_nr
41863526108385   1             70206
135153013040192  1             70659
161514654074162  1            248758
186034136943920  1             74393
                 2             37583
Name: secs_since_last_arrival, dtype: int64

In [286]:
devices = devices.merge(time, how='left', left_index=True, right_index=True)

In [287]:
devices.head()

,,secs_to_next_mean,secs_since_last_arrival
ref_hash,window_nr,,
2564673204772915246,1,765.0,88625.0
4441121667607578179,1,111.0,29052.0
7721769811471055264,1,281.0,333.0
6416039086842158968,1,NaN,NaN
1258642015983312729,1,8365.0,121224.0


### Cantidad de apariciones en encuestas en la ventana

In [288]:
%%time
time = auctions.groupby(group ,sort=False)['n'].sum().rename('auctions_total')

CPU times: user 2.28 s, sys: 972 ms, total: 3.25 s
Wall time: 3.25 s


In [289]:
time.head()

ref_hash         window_nr
41863526108385   1            34
135153013040192  1             7
161514654074162  1             5
186034136943920  1             6
                 2            58
Name: auctions_total, dtype: int64

In [290]:
devices = devices.merge(time, how='left', left_index=True, right_index=True)

In [291]:
devices.head()

,,secs_to_next_mean,secs_since_last_arrival,auctions_total
ref_hash,window_nr,,,
2564673204772915246,1,765.0,88625.0,216.0
4441121667607578179,1,111.0,29052.0,1188.0
7721769811471055264,1,281.0,333.0,908.0
6416039086842158968,1,NaN,NaN,NaN
1258642015983312729,1,8365.0,121224.0,12.0


### Cantidad de apariciones en encuestas en la ultima hora

In [292]:
%%time
time = auctions.merge(max_dates, on='window_nr', how='left')
time = time.loc[time['created'] >= (time['w_max_date'] - timedelta(hours=1))]\
           .groupby(group, sort=False)['n'].sum()\
           .rename('auctions_last_hour')

CPU times: user 14.9 s, sys: 11 s, total: 26 s
Wall time: 3min 27s


In [293]:
time.head()

ref_hash         window_nr
345999128501141  2             3
360710529886978  3             2
416301579449694  1             2
686608884458246  3            11
717556230663455  1            74
Name: auctions_last_hour, dtype: int64

In [294]:
devices = devices.merge(time, how='left', left_index=True, right_index=True)

In [295]:
devices.head()

,,secs_to_next_mean,secs_since_last_arrival,auctions_total,auctions_last_hour
ref_hash,window_nr,,,,
2564673204772915246,1,765.0,88625.0,216.0,NaN
4441121667607578179,1,111.0,29052.0,1188.0,NaN
7721769811471055264,1,281.0,333.0,908.0,23.0
6416039086842158968,1,NaN,NaN,NaN,NaN
1258642015983312729,1,8365.0,121224.0,12.0,NaN


### Cantidad de apariciones en encuestas en el fin de semana

In [296]:
%%time
weekend = [20,21]
time = auctions.loc[(auctions['created'].dt.day).isin(weekend)]\
               .groupby(group,sort=False)['n'].sum()\
               .rename('amount_auctions_in_weekend')

CPU times: user 3.91 s, sys: 2.66 s, total: 6.57 s
Wall time: 37 s


In [297]:
time.head()

ref_hash         window_nr
41863526108385   1            33
135153013040192  1             7
186034136943920  1             4
                 2            11
360710529886978  1             2
Name: amount_auctions_in_weekend, dtype: int64

In [298]:
devices = devices.merge(time, how='left', left_index=True, right_index=True)

In [299]:
devices.head()

,,secs_to_next_mean,secs_since_last_arrival,auctions_total,auctions_last_hour,amount_auctions_in_weekend
ref_hash,window_nr,,,,,
2564673204772915246,1,765.0,88625.0,216.0,NaN,NaN
4441121667607578179,1,111.0,29052.0,1188.0,NaN,640.0
7721769811471055264,1,281.0,333.0,908.0,23.0,218.0
6416039086842158968,1,NaN,NaN,NaN,NaN,NaN
1258642015983312729,1,8365.0,121224.0,12.0,NaN,NaN


### Ultima aparicion es weekend

In [300]:
weekend = [20,21]
time = auctions.groupby(group, sort=False)\
               .tail(1)\
               .set_index(group)['created'].transform(lambda x: x.day in weekend)\
               .rename('is_last_weekend')

In [301]:
time.head()

ref_hash         window_nr
41863526108385   1             True
135153013040192  1             True
161514654074162  1            False
186034136943920  1             True
                 2            False
Name: is_last_weekend, dtype: bool

In [302]:
devices = devices.merge(time, how='left', left_index=True, right_index=True)

In [303]:
devices.head()

,,secs_to_next_mean,secs_since_last_arrival,auctions_total,auctions_last_hour,amount_auctions_in_weekend,is_last_weekend
ref_hash,window_nr,,,,,,
2564673204772915246,1,765.0,88625.0,216.0,NaN,NaN,False
4441121667607578179,1,111.0,29052.0,1188.0,NaN,640.0,True
7721769811471055264,1,281.0,333.0,908.0,23.0,218.0,True
6416039086842158968,1,NaN,NaN,NaN,NaN,NaN,NaN
1258642015983312729,1,8365.0,121224.0,12.0,NaN,NaN,False


### Cantidad de sources distintos

In [304]:
temp = auctions.groupby(by = group +['source_id'], sort=False).agg({'n':'sum'}).reset_index()
temp['n'] = 1
temp = temp.groupby(by = group).agg({'n' : 'sum'})
temp.rename(columns = {'n': 'amount_dif_src' },inplace = True)

In [305]:
temp.head()

amount_dif_src
ref_hash        window_nr                
41863526108385  1                       3
135153013040192 1                       1
161514654074162 1                       2
186034136943920 1                       1
                2                       1

In [306]:
devices = devices.merge(temp, how = 'left', left_index=True, right_index=True)

In [307]:
devices.head()

,,secs_to_next_mean,secs_since_last_arrival,auctions_total,auctions_last_hour,amount_auctions_in_weekend,is_last_weekend,amount_dif_src
ref_hash,window_nr,,,,,,,
2564673204772915246,1,765.0,88625.0,216.0,NaN,NaN,False,3.0
4441121667607578179,1,111.0,29052.0,1188.0,NaN,640.0,True,5.0
7721769811471055264,1,281.0,333.0,908.0,23.0,218.0,True,3.0
6416039086842158968,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1258642015983312729,1,8365.0,121224.0,12.0,NaN,NaN,False,1.0


## Cantidad de eventos

In [308]:
temp = events.groupby(group, sort=False)\
             .agg({'n':'sum'})\
             .rename(columns = {'n': 'amount_events' })

In [309]:
temp.head()

,,amount_events
ref_hash,window_nr,
4314242242878368557,1,10
2068670382964280534,1,140
726218951290512705,1,27
1329210878469536477,1,157
5661732594213028190,1,50


In [310]:
devices = devices.merge(temp, how = 'left', left_index=True, right_index=True)

In [311]:
devices.head()

,,secs_to_next_mean,secs_since_last_arrival,auctions_total,auctions_last_hour,amount_auctions_in_weekend,is_last_weekend,amount_dif_src,amount_events
ref_hash,window_nr,,,,,,,,
2564673204772915246,1,765.0,88625.0,216.0,NaN,NaN,False,3.0,NaN
4441121667607578179,1,111.0,29052.0,1188.0,NaN,640.0,True,5.0,NaN
7721769811471055264,1,281.0,333.0,908.0,23.0,218.0,True,3.0,99.0
6416039086842158968,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1258642015983312729,1,8365.0,121224.0,12.0,NaN,NaN,False,1.0,NaN


### Ultimo evento

In [312]:
temp = events.groupby(group,sort=False)\
             .agg({'created':'max'})\
             .reset_index()\
             .merge(max_dates, on='window_nr', how='left')\
             .set_index(group)
temp = (temp['w_max_date'] - temp['created']).transform(lambda x: round(x.total_seconds())).rename('secs_since_last_event')

In [313]:
temp.head()

ref_hash             window_nr
4314242242878368557  1             14186
2068670382964280534  1             15512
726218951290512705   1            210044
1329210878469536477  1              1245
5661732594213028190  1             15836
Name: secs_since_last_event, dtype: int64

In [314]:
devices = devices.merge(temp, how = 'left', left_index=True, right_index=True)

In [315]:
devices.head()

,,secs_to_next_mean,secs_since_last_arrival,auctions_total,auctions_last_hour,amount_auctions_in_weekend,is_last_weekend,amount_dif_src,amount_events,secs_since_last_event
ref_hash,window_nr,,,,,,,,,
2564673204772915246,1,765.0,88625.0,216.0,NaN,NaN,False,3.0,NaN,NaN
4441121667607578179,1,111.0,29052.0,1188.0,NaN,640.0,True,5.0,NaN,NaN
7721769811471055264,1,281.0,333.0,908.0,23.0,218.0,True,3.0,99.0,196475.0
6416039086842158968,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1258642015983312729,1,8365.0,121224.0,12.0,NaN,NaN,False,1.0,NaN,NaN


### Alguno de los eventos fueron con wifi?

In [316]:
temp = events.groupby(group, sort=False).agg({'wifi':'sum'})
temp['wifi'] = temp['wifi'].astype(bool)

In [317]:
temp.head()

,,wifi
ref_hash,window_nr,
4314242242878368557,1,True
2068670382964280534,1,True
726218951290512705,1,True
1329210878469536477,1,True
5661732594213028190,1,False


In [318]:
devices = devices.merge(temp, how = 'left', left_index=True, right_index=True)
#devices['wifi'].fillna(?)

In [319]:
devices.head()

,,secs_to_next_mean,secs_since_last_arrival,auctions_total,auctions_last_hour,amount_auctions_in_weekend,is_last_weekend,amount_dif_src,amount_events,secs_since_last_event,wifi
ref_hash,window_nr,,,,,,,,,,
2564673204772915246,1,765.0,88625.0,216.0,NaN,NaN,False,3.0,NaN,NaN,NaN
4441121667607578179,1,111.0,29052.0,1188.0,NaN,640.0,True,5.0,NaN,NaN,NaN
7721769811471055264,1,281.0,333.0,908.0,23.0,218.0,True,3.0,99.0,196475.0,False
6416039086842158968,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1258642015983312729,1,8365.0,121224.0,12.0,NaN,NaN,False,1.0,NaN,NaN,NaN


### Tiempo promedio de clicks

In [320]:
temp = clicks.groupby(by = ['ref_hash', 'window_nr'])\
             .agg({'timeToClick' : 'mean'})\
             .rename(columns = {'timeToClick' : 'timeToClick_mean'})

In [321]:
devices = devices.merge(temp, how = 'left', left_index=True, right_index=True)
#devices['timeToClick_mean'].fillna(?)

### Cantidad de advertiser que tiene cada ref_hash

In [322]:
temp = clicks.groupby(group + ['advertiser_id'],sort=False).agg({'n':'sum'}).reset_index()
temp['n'] = 1
temp = temp.groupby(group)\
           .agg({'n' : 'sum'})\
           .rename(columns = {'n': 'amount_dif_advertisers' })

In [323]:
temp.head()

,,amount_dif_advertisers
ref_hash,window_nr,
693609737448534,2.0,1
1461247282174365,1.0,1
2204225481747532,2.0,1
5470466329076030,1.0,1
5534899846690585,1.0,1


In [324]:
devices = devices.merge(temp, how = 'left', left_index=True, right_index=True)

---
## 4. Guardo Features 
Aplico arreglo: 
* Si el dispositivo no aparecio en la ventana, se elimina la fila
* Si el dispositivo no tiene valores en los eventos de count agrego count 0

In [325]:
#Chequear casos que queden con medias en null
devices = devices.dropna(how='all')

In [326]:
devices.isnull().sum()

secs_to_next_mean             390198
secs_since_last_arrival       390198
auctions_total                390198
auctions_last_hour            942576
amount_auctions_in_weekend    729211
is_last_weekend               390198
amount_dif_src                390198
amount_events                 463802
secs_since_last_event         463802
wifi                          463802
timeToClick_mean              987136
amount_dif_advertisers        976314
dtype: int64

In [327]:
devices.dtypes

secs_to_next_mean             float64
secs_since_last_arrival       float64
auctions_total                float64
auctions_last_hour            float64
amount_auctions_in_weekend    float64
is_last_weekend                object
amount_dif_src                float64
amount_events                 float64
secs_since_last_event         float64
wifi                           object
timeToClick_mean              float64
amount_dif_advertisers        float64
dtype: object

In [328]:
#Fix means
means = [x for x in devices.columns if 'mean' in x]
for i in means:
    print("Filling ",i, " with window mean")
    devices[i] = devices.groupby('window_nr')[i].transform(lambda x: x.fillna(x.mean()))

#Fix Strings
obj = list(devices.dtypes[devices.dtypes == 'object'].index)
for i in obj:
    print("Filling ",i, " with False")
    devices[i] = devices[i].fillna(False)

print('Filling the rest with 0')
devices = devices.fillna(0)

Filling  secs_to_next_mean  with window mean
Filling  timeToClick_mean  with window mean
Filling  is_last_weekend  with False
Filling  wifi  with False
Filling the rest with 0


In [329]:
devices.isnull().sum()

secs_to_next_mean             0
secs_since_last_arrival       0
auctions_total                0
auctions_last_hour            0
amount_auctions_in_weekend    0
is_last_weekend               0
amount_dif_src                0
amount_events                 0
secs_since_last_event         0
wifi                          0
timeToClick_mean              0
amount_dif_advertisers        0
dtype: int64

In [330]:
devices.to_csv('training_set.csv',header=True,index=True)

In [337]:
to_predict['ref_hash'].isin(auctions.loc[auctions['window_nr'] == 1]['ref_hash']).agg([np.sum,'count'])

sum      1916
count    4037
Name: ref_hash, dtype: int64